In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
review_json_path = '../raw_data/yelp_academic_dataset_review.json'

In [3]:
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [4]:
def get_bus_data():
    df = pd.read_json('../raw_data/yelp_academic_dataset_business.json', lines=True)
    df_restaurants = df[df['is_open']==1]
    df_restaurants = df_restaurants[df_restaurants.categories.notna()]
    df_restaurants = df_restaurants[df_restaurants.categories.str.contains("Restaurants")]

    df_rest_filter = df_restaurants[(df_restaurants.city == 'Boston') | (df_restaurants.city == 'Westerville')]

    return df_rest_filter
df_bus = get_bus_data()

In [5]:
chunk_list = []
for chunk_review in review:
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    chunk_merged = pd.merge(df_bus, chunk_review, on='business_id', how='inner')
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)

df_review = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

52356 out of 1,000,000 related reviews
47037 out of 1,000,000 related reviews
43727 out of 1,000,000 related reviews
45505 out of 1,000,000 related reviews
42311 out of 1,000,000 related reviews
46892 out of 1,000,000 related reviews
49242 out of 1,000,000 related reviews
44836 out of 1,000,000 related reviews
35560 out of 1,000,000 related reviews


In [8]:
df_review = df_review[['name','city','review_stars','text']]